# P2 

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.nn.functional as F

from torchvision import models
from torch import optim
from sklearn.metrics import f1_score

In [ ]:
class DWSepConv2d(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels, ker, padd, bias=True):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=ker, stride=1, padding=padd)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size= ker, stride=1, padding=padd)
        
    def forward(self, xb):
        xb = xb.view(-1, 3, 224, 224)
        xb = self.depthwise(xb.float())
        xb = F.relu(self.pointwise(xb))
        return xb.view(-1, xb.size(1))

In [ ]:
class VGG16DWSep(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv4 = DWSepConv2d(64, 128, ker=3, padd=1)
        self.conv5 = DWSepConv2d(128, 128, ker=3, padd=1)
        self.conv6 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv7 = DWSepConv2d(128, 256, ker=3, padd=1)
        self.conv8 = nn.BatchNorm2d(256)
        self.conv9 = DWSepConv2d(256, 256, ker=3, padd=1)
        self.conv10 = nn.BatchNorm2d(256)
        self.conv11 = DWSepConv2d(256, 256, ker=3, padd=1)
        self.conv12 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv13 = DWSepConv2d(256, 512, ker=3, padd=1)
        self.conv14 = nn.BatchNorm2d(512)
        self.conv15 = DWSepConv2d(512, 512, ker=3, padd=1)
        self.conv16 = nn.BatchNorm2d(512)
        self.conv17 = DWSepConv2d(512, 512, ker=3, padd=1)
        self.conv18 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv19 = nn.Flatten()
        self.conv20 = nn.Linear(512, 1024)
        self.conv21 = nn.Dropout(0.7)
        self.conv22 = nn.Linear(1024, 512)
        self.conv23 = nn.Dropout(0.5)
        self.conv24 = nn.Linear(512, 2)
        
    
    def forward(self, xb):
        xb = xb.view(-1, 3, 224, 224)
        xb = F.relu(self.conv1(xb.float()))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.relu(self.conv4(xb))
        xb = F.relu(self.conv5(xb))
        xb = F.relu(self.conv6(xb))
        xb = F.relu(self.conv7(xb))
        xb = F.relu(self.conv8(xb))
        xb = F.relu(self.conv9(xb))
        xb = F.relu(self.conv10(xb))
        xb = F.relu(self.conv11(xb))
        xb = F.relu(self.conv12(xb))
        xb = F.relu(self.conv13(xb))
        xb = F.relu(self.conv14(xb))
        xb = F.relu(self.conv15(xb))
        xb = F.relu(self.conv16(xb))
        xb = F.relu(self.conv17(xb))
        xb = F.relu(self.conv18(xb))
        xb = F.relu(self.conv19(xb))
        xb = F.relu(self.conv20(xb))
        xb = F.relu(self.conv21(xb))
        xb = F.relu(self.conv22(xb))
        xb = F.relu(self.conv23(xb))
        xb = F.relu(self.conv24(xb))
        return xb.view(-1, xb.size(1))
        

In [ ]:
#Poner pesos de capa pre entrenada

VGG16 = models.vgg16(pretrained=True, progress=True)

pesos1 = VGG16.features[0].weight
pesos1.requires_grad = False

pesos2 = VGG16.features[2].weight
pesos2.requires_grad = False

In [ ]:
# transferencia de pesos

class VGG16DWSep(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv1.weight = pesos1
        
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv2.weight = pesos2
        
        self.conv3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv4 = DWSepConv2d(64, 128, ker=3, padd=1)
        self.conv5 = DWSepConv2d(128, 128, ker=3, padd=1)
        self.conv6 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv7 = DWSepConv2d(128, 256, ker=3, padd=1)
        self.conv8 = nn.BatchNorm2d(256)
        self.conv9 = DWSepConv2d(256, 256, ker=3, padd=1)
        self.conv10 = nn.BatchNorm2d(256)
        self.conv11 = DWSepConv2d(256, 256, ker=3, padd=1)
        self.conv12 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv13 = DWSepConv2d(256, 512, ker=3, padd=1)
        self.conv14 = nn.BatchNorm2d(512)
        self.conv15 = DWSepConv2d(512, 512, ker=3, padd=1)
        self.conv16 = nn.BatchNorm2d(512)
        self.conv17 = DWSepConv2d(512, 512, ker=3, padd=1)
        self.conv18 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv19 = nn.Flatten()
        self.conv20 = nn.Linear(512*81, 1024)
        self.conv21 = nn.Dropout(0.7)
        self.conv22 = nn.Linear(1024, 512)
        self.conv23 = nn.Dropout(0.5)
        self.conv24 = nn.Linear(512, 2)
        
    
    def forward(self, xb):
        xb = xb.view(-1, 3, 224, 224)
        xb = F.relu(self.conv1(xb.float()))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.relu(self.conv4(xb))
        xb = F.relu(self.conv5(xb))
        xb = F.relu(self.conv6(xb))
        xb = F.relu(self.conv7(xb))
        xb = F.relu(self.conv8(xb))
        xb = F.relu(self.conv9(xb))
        xb = F.relu(self.conv10(xb))
        xb = F.relu(self.conv11(xb))
        xb = F.relu(self.conv12(xb))
        xb = F.relu(self.conv13(xb))
        xb = F.relu(self.conv14(xb))
        xb = F.relu(self.conv15(xb))
        xb = F.relu(self.conv16(xb))
        xb = F.relu(self.conv17(xb))
        xb = F.relu(self.conv18(xb))
        xb = F.relu(self.conv19(xb))
        xb = F.relu(self.conv20(xb))
        xb = F.relu(self.conv21(xb))
        xb = F.relu(self.conv22(xb))
        xb = F.relu(self.conv23(xb))
        xb = F.relu(self.conv24(xb))
        return xb.view(-1, xb.size(1))
        

In [ ]:
class EarlyStopping():
    
    def __init__(self, modo='min', paciencia=5, porcentaje=False, tol=0):
        self.modo = modo
        self.paciencia = paciencia
        self.porcentaje = porcentaje
        self.tol = tol
        self.best = None
        self.stopstep = 0
        
    def mejor(self, metrica_validacion):
        
        if self.best == None:
            self.best = metrica_validacion
            return True
        
        elif self.porcentaje:
            if self.modo == 'min':
                if metrica_validacion < self.best*(1-self.tol):
                    self.best = metrica_validacion
                    self.stopstep = 0 
                    return True
                else:
                    self.stopstep += 1
                    return False
            else:
                if metrica_validacion > self.best*(1+self.tol):
                    self.best = metrica_validacion
                    self.stopstep = 0
                    return True
                else: 
                    self.stopstep += 1
                    return False
                
        else:
            if self.modo == 'min':
                if metrica_validacion < self.best - self.tol:
                    self.best = metrica_validacion
                    self.stopstep = 0
                    return True
                else:
                    self.stopstep += 1
                    return False
            else:
                if metrica_validacion > self.best + self.tol:
                    self.best = metrica_validacion
                    self.stopstep = 0
                    return True
                else:
                    self.stopstep += 1
                    return False

    def deberia_parar(self, metrica_validacion):
        if mejor(self, metrica_validacion):
            return False
        elif self.stopstep < self.paciencia:
            return False
        else:
            return True
        
    

In [ ]:
torch.cuda.is_available()

In [ ]:
num_epochs = 8

es = EarlyStopping(modo = 'max')

f1_list = []
accuracy = []
losses = []

model = VGG16DWSep()
model.cuda()
opt = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_func = F.cross_entropy

for epoch in range(num_epochs):
    
    #ciclo de entrenamiento
    for i, (images, labels) in enumerate(loader_train):

        outputs = model(images.cuda())
        loss = loss_func(outputs.cuda(), labels.long().cuda())
        losses.append(loss.item())

        loss.backward()
        opt.step()
        opt.zero_grad()
    
    
    #ciclo de validacion
    right = 0
    total = 0
    for i, (images, labels) in enumerate(loader_val):
        outputs = model(images.cuda()) 
        correct += (torch.argmax(outputs, dim=1) == labels.cuda()).float().sum() 
        total += len(labels)
    
    metrica_validacion_1 = 100 * right / total
    metrica_validacion_1 = metrica_validacion_1.item()
    
    acurracy.append(metrica_validacion_1)
    
    if es.deberia_parar(metrica_validacion):
        #se cumple el criterio de early stop
        break